# Tracking football motion

## Importing the Libs

In [1]:
# python libraries
from collections import deque
import os
import time

In [2]:
# third party libraries
import cv2
from ipywebrtc import Video
import numpy as np

## Creating global objects

In [27]:
lower_color = (170, 100, 20)
upper_color = (200, 255, 255)
buffer = 64
pts = deque(maxlen=buffer)

In [28]:
classes = None

with open('coco-names.txt', 'r') as f:
    classes = [line.strip() for line in f.readlines()]

Width = None
Height = None
net = cv2.dnn.readNetFromDarknet('yolov3-tiny.cfg', 'yolov3-tiny.weights')
ln = net.getLayerNames()
ln = [ln[i[0] - 1] for i in net.getUnconnectedOutLayers()]

## Grabbing our Video

In [22]:
video = Video.from_file('untracked_football.mp4')
video

Video(value=b'\x00\x00\x00\x1cftypmp42\x00\x00\x00\x01mp41mp42isom\x00\x00\x00\x08wide\x00\x1e\xb6wmdat!\x1a\x…

In [30]:
cap = cv2.VideoCapture('untracked_football.mp4')

## Detecting the football

In [31]:
def detect(frame):
    # Color method
    center = 0, 0
    radius = 0
    kernel = None
    
    blurred = cv2.GaussianBlur(frame, (3, 3), 0)
    hsv = cv2.cvtColor(blurred, cv2.COLOR_BGR2HSV)
    
    mask = cv2.inRange(hsv,  lower_color, upper_color)
    mask = cv2.erode(mask, kernel, iterations=3)
    mask = cv2.dilate(mask, kernel, iterations=3)
    
    _, contours, _ = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    if len(contours) > 0:
        max_contour = max(contours, key=cv2.contourArea)
        ((x, y), radius) = cv2.minEnclosingCircle(max_contour)
        center = int(x), int(y)
    
    if radius > 10:
        cv2.circle(frame, center, int(radius), (0, 255, 255), 2)
        cv2.circle(frame, center, 5, (0, 0, 255), -1)
        pts.appendleft(center)
    return frame

In [ ]:
def detect(frame):
    # Neural Network method
    blob = cv2.dnn.blobFromImage(frame, .1, (416, 416), (0,0,0), swapRB=True, crop=False)
    net.setInput(blob)
    layer_outputs = net.forward(ln)
    
    boxes = []
    confidences = []
    class_ids = []
    
    for output in layer_outputs:
        for detection in output:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            
            if confidence > 0.5:
                center_x = int(detection[0] * Width)
                center_y = int(detection[1] * Height)
                
                w = int(detection[2] * Width)
                h = int(detection[3] * Height)
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)
                
                class_ids.append(class_id)
                confidences.append(float(confidence))
                boxes.append([x, y, w, h])

    indices = cv2.dnn.NMSBoxes(boxes, confidences, .5, .4)
    
    for i in indices:
        i = i[0]
        box = boxes[i]
        x = box[0]
        y = box[1]
        w = box[2]
        h = box[3]
        
        label = str(classes[class_ids[i]])
        
#         if label != 'sports ball':
#             continue
        
        cv2.rectangle(frame, (x, y), ((x + w), (y + h)), (0, 255, 255), 2)
        cv2.putText(frame, label, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2)
    return frame


## Tracking the football

In [32]:
while True:
    # grab the current frame
    grabbed, frame = cap.read()
    
    # if we are viewing a video and we did not grab a frame,
    # then we have reached the end of the video
    if not grabbed:
        break
    
    if Width is None or Height is None:
        Width, Height = frame.shape[:2]

    frame = detect(frame)

    if frame is None:
        continue

    # loop over the set of tracked points
    for i in range(1, len(pts)):
        if pts[i - 1] is None or pts[i] is None:
            continue

        thickness = int(np.sqrt(len(pts) / float(i + 1)) * 2.5)
        cv2.line(frame, pts[i - 1], pts[i], (0, 0, 255), thickness)

    # show the frame to our screen
    cv2.imshow("Frame", frame)
    key = cv2.waitKey(1) & 0xFF

    # if the 'q' key is pressed, stop the loop
    if key == ord("q"):
        break

In [26]:
# close all windows, release video_file, and empty deque
cap.release()
cv2.destroyAllWindows()
pts.clear()